In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, GPT2LMHeadModel, PreTrainedTokenizerFast

In [ ]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'
DATA_PATH = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'
SUBMISSION_PATH = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'
SEED = 42
reset_seeds(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
# 데이터 로드
train = pd.read_csv(f'{DATA_PATH}category_train_120000_ver1.csv')

train.shape

(120000, 5)

In [ ]:
model_name = 'skt/kogpt2-base-v2'

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
train

,category,question,answer,qna,qna_category
0,['건축구조'],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,면진장치가 뭐야? 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동...,['설명']
1,['건축구조'],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계의 종류 좀 알려줘 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.",['종류']
2,['건축구조'],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점이 뭐야? 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문...,['장점']
3,['건축구조'],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근 콘크리트 구조가 뭐야? 철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 ...",['설명']
4,['건축구조'],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 어떤 방식이 있어? 철골구조는 일반철골구조와 경량철골구조가 있습니다.,['종류']
...,...,...,...,...,...
119995,"['기타', '시공']","새집증후군이 무엇이며, 외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼?","새집증후군이란 주택, 및 건축물 등 생활환경 속에서 일어나는 화학물질 과민증으로 신...","새집증후군이 무엇이며, 외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼?...","['설명', '확인']"
119996,"['마감재', '인테리어']","써모사이딩이 무엇이며, 인테리어 디자인에서 조명의 역할은 무엇인가요?",써모사이딩은 메탈사이딩이라고도 불리는 외장재이며 알루미늄 합금 강판에 고내후성 도료...,"써모사이딩이 무엇이며, 인테리어 디자인에서 조명의 역할은 무엇인가요? 써모사이딩은 ...",['설명']
119997,['마감하자'],"훼손이 무엇이고, 공간 내 높은습도로 인해 도배지에 얼룩이 발생할 수 있어?","훼손은 터짐 하자, 몰딩수정 등 물리적인 충격이나 힘에 의해 발생하는 하자입니다. ...","훼손이 무엇이고, 공간 내 높은습도로 인해 도배지에 얼룩이 발생할 수 있어? 훼손은...","['설명', '가능성']"
119998,"['기타', '마감하자']","새집증후군의 원인이 무엇이고, 벽지에 반점이 생기는 원인이 뭐야?",새집증후군은 새솔 지은 집에 입주했을 때 이전에 없던 이상 증상이 신체에 나타나는 ...,"새집증후군의 원인이 무엇이고, 벽지에 반점이 생기는 원인이 뭐야? 새집증후군은 새솔...",['원인']


In [ ]:
qna_data = []
for _, row in tqdm(train.iterrows(), total=train.shape[0]):
    input_text = "<q>" + row['question'] + "</s><a>" + row['answer'] + "</s><c>" + row['category'] + "</s><t>" + row['qna_category'] + "</s>"
    qna_data.append(input_text)

qna_df = pd.DataFrame({'qna':qna_data})
qna_df

  0%|          | 0/120000 [00:00<?, ?it/s]

,qna
0,<q>면진장치가 뭐야?</s><a>면진장치란 지반에서 오는 진동 에너지를 흡수하여 ...
1,"<q>내진설계의 종류 좀 알려줘</s><a>내진 설계의 종류로 내진구조, 제진구조,..."
2,<q>철골구조의 장점이 뭐야?</s><a>철골구조는 건물의 외벽에는 그다지 하중이 ...
3,"<q>철골철근 콘크리트 구조가 뭐야?</s><a>철근철골콘크리트는 철골과 철근, 그..."
4,<q>철골구조는 어떤 방식이 있어?</s><a>철골구조는 일반철골구조와 경량철골구조...
...,...
119995,"<q>새집증후군이 무엇이며, 외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도..."
119996,"<q>써모사이딩이 무엇이며, 인테리어 디자인에서 조명의 역할은 무엇인가요?</s><..."
119997,"<q>훼손이 무엇이고, 공간 내 높은습도로 인해 도배지에 얼룩이 발생할 수 있어?<..."
119998,"<q>새집증후군의 원인이 무엇이고, 벽지에 반점이 생기는 원인이 뭐야?</s><a>..."


In [ ]:
class DobaeDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.question = df['qna'].tolist()

    def __len__(self):
        return len(self.question)

    def __getitem__(self, idx):
        return self.question[idx]

In [ ]:
def collate_fn(batch):
    x = tokenizer(batch, return_tensors='pt', padding=True)
    return {'x': x}

In [ ]:
dt = DobaeDataset(qna_df)
dl = torch.utils.data.DataLoader(dt, batch_size=2, collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   455,   405,  7532,  8265, 37765, 46651,  7991,   406,     1,
           9724,   439,   405,  7532,  8265, 20725,  7374,  9027,  7599,  9023,
          14472, 15898, 14820, 33220, 36928, 10764,  9166, 11818, 28037, 10090,
          15898, 34062, 20725, 21154,     1,  9724,   441,   405,   433,   382,
          13736, 12791,   382,   435,     1,  9724,   458,   405,   433,   382,
          16623,   382,   435,     1],
         [ 9724,   455,   405,  7071,  8265,  7793, 10346, 10036, 11732,  9666,
           8244,     1,  9724,   439,   405,  7071,  8265,  9160, 10346, 32032,
           9094,  8265, 12791,   387,  9037,  8265, 12791,   387,  9411,  8265,
          27211, 32987,     1,  9724,   441,   405,   433,   382, 13736, 12791,
            382,   435,     1,  9724,   458,   405,   433,   382, 13021,   382,
            435,     1,     3,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        pred = model(**x).logits
        n_class = pred.shape[-1]
        pred = pred[:, :-1, :]
        pred = pred.reshape(-1, n_class)

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()

        mask = tgt != tokenizer.pad_token_id
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [ ]:
batch_size = 6
lr = 3e-5
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 10

In [ ]:
reset_seeds(SEED)

model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

train_dt = DobaeDataset(qna_df)
train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

for i in tqdm(range(epochs)):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(f'{i+1}번째 epoch: {train_loss:.5f}')

    model.save_pretrained(f'{DATA_PATH}model/kogpt2_category_1_0310{qna_df.shape[0]}_{i+1}epoch_{train_loss:.5f}loss')

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

1번째 epoch: 0.38773


...

7번째 epoch: 0.13602


  0%|          | 0/20000 [00:00<?, ?it/s]

8번째 epoch: 0.13355


  0%|          | 0/20000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from google.colab import runtime

In [ ]:
runtime.unassign()